# 🌌 x.titan: Autonomous Trading Brain - Kaggle Training

**Version**: 2026.1.10
**Source**: GitHub → https://github.com/planetazul3/x.titan

## 1. Clone Repository

In [ ]:
import os, sys
from pathlib import Path

WORKING_DIR = Path('/kaggle/working/x.titan')
if not WORKING_DIR.exists():
    print('📥 Cloning x.titan...')
    !git clone --depth 1 https://github.com/planetazul3/x.titan.git {WORKING_DIR}
else:
    print('✅ Repo exists')

os.chdir(WORKING_DIR)
sys.path.insert(0, str(WORKING_DIR))
print(f'📂 {os.getcwd()}')

## 2. TA-Lib

In [ ]:
%%bash
set -e
if [ ! -f /usr/include/ta-lib/ta_defs.h ]; then
    echo '📦 Installing TA-Lib...'
    apt-get update -qq && apt-get install -y -qq build-essential wget
    wget -q http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
    tar -xzf ta-lib-0.4.0-src.tar.gz && cd ta-lib
    ./configure --prefix=/usr > /dev/null && make > /dev/null && make install > /dev/null
    cd .. && rm -rf ta-lib ta-lib-0.4.0-src.tar.gz
    echo '✅ TA-Lib compiled'
else
    echo '✅ TA-Lib already installed'
fi

## 3. Python Dependencies

In [ ]:
print('📥 Installing packages...')
!pip install -q TA-Lib pandas numpy torch tqdm pydantic pydantic-settings python-dotenv pyarrow pandera

import subprocess
result = subprocess.run(['pip', 'install', '-q', '-e', './python-deriv-api'], capture_output=True)
if result.returncode != 0:
    !pip install -q git+https://github.com/planetazul3/python-deriv-api.git

import talib, deriv_api, pandera
print('✅ Dependencies installed')

## 4. Environment Configuration

In [ ]:
# Load API token (optional)
DERIV_API_TOKEN = ''
try:
    from kaggle_secrets import UserSecretsClient
    DERIV_API_TOKEN = UserSecretsClient().get_secret('DERIV_API_TOKEN') or ''
    if DERIV_API_TOKEN: print('✅ DERIV_API_TOKEN loaded')
except: pass

# FULL .env with ALL required fields
env_content = f'''ENVIRONMENT=test
DERIV_API_TOKEN={DERIV_API_TOKEN}
DERIV_APP_ID=1089

# Trading (REQUIRED)
TRADING__SYMBOL=R_100
TRADING__STAKE_AMOUNT=10.0
TRADING__TIMEFRAME=1m

# Safety
EXECUTION_SAFETY__KILL_SWITCH_ENABLED=false
EXECUTION_SAFETY__MAX_DAILY_LOSS=100.0
EXECUTION_SAFETY__MAX_STAKE_PER_TRADE=10.0

# Thresholds (REQUIRED)
THRESHOLDS__CONFIDENCE_THRESHOLD_HIGH=0.80
THRESHOLDS__LEARNING_THRESHOLD_MIN=0.50
THRESHOLDS__LEARNING_THRESHOLD_MAX=0.70
THRESHOLDS__CAUTION_THRESHOLD_MARGIN=0.05

# Hyperparams
HYPERPARAMS__USE_TFT=True
HYPERPARAMS__LEARNING_RATE=0.0007
HYPERPARAMS__BATCH_SIZE=128
HYPERPARAMS__LSTM_HIDDEN_SIZE=256
HYPERPARAMS__CNN_FILTERS=128
HYPERPARAMS__LATENT_DIM=64
HYPERPARAMS__DROPOUT_RATE=0.2
HYPERPARAMS__EWC_SAMPLE_SIZE=2000

# Data shapes
DATA_SHAPES__SEQUENCE_LENGTH_TICKS=1000
DATA_SHAPES__SEQUENCE_LENGTH_CANDLES=200
DATA_SHAPES__WARMUP_STEPS=50
'''
with open('.env', 'w') as f:
    f.write(env_content)
print('✅ .env created')

## 5. GPU Detection

In [ ]:
import torch

# Conservative batch sizes to avoid OOM on T4 (16GB)
BATCH_SIZE = 16
if torch.cuda.is_available():
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f'✅ GPU: {torch.cuda.get_device_name(0)} ({gpu_mem:.1f} GB)')
    # OOM fix: T4 with 15GB -> batch 128 max (not 256)
    BATCH_SIZE = 128 if gpu_mem >= 15 else 64 if gpu_mem >= 8 else 32
    print(f'   Batch size: {BATCH_SIZE}')
else:
    print('⚠️ NO GPU')

os.environ['HYPERPARAMS__BATCH_SIZE'] = str(BATCH_SIZE)

## 6. Data Setup

In [ ]:
# Download from GitHub Release
DATA_PATH = Path('./data_cache')

if not DATA_PATH.exists():
    print('📥 Downloading training data from GitHub Release...')
    !wget -q https://github.com/planetazul3/x.titan/releases/download/v2026.1.8-data/xtitan_training_data_2025.zip
    !unzip -q xtitan_training_data_2025.zip
    !rm xtitan_training_data_2025.zip

tick_files = list(DATA_PATH.glob('**/ticks/*.parquet'))
candle_files = list(DATA_PATH.glob('**/candles*/*.parquet'))

if len(tick_files) >= 6 and len(candle_files) >= 6:
    print(f'✅ Data ready: {len(tick_files)} tick files, {len(candle_files)} candle files')
    DATA_READY = True
else:
    print('❌ Data extraction failed')
    DATA_READY = False

## 7. Training

In [ ]:
import time

if not DATA_READY:
    print('❌ SKIPPING - No data')
else:
    Path('checkpoints').mkdir(exist_ok=True)
    print('🚀 STARTING TRAINING')
    start = time.time()
    
    !python scripts/train.py \
        --data-path data_cache \
        --epochs 30 \
        --batch-size {BATCH_SIZE} \
        --checkpoint-dir checkpoints \
        --months 12
    
    print(f'\n⏱️ Completed in {(time.time()-start)/60:.1f} min')

## 8. Export Model

In [ ]:
import shutil
from datetime import datetime

best_model = Path('checkpoints/best_model.pt')

if best_model.exists():
    print(f'✅ best_model.pt: {best_model.stat().st_size/1e6:.1f} MB')
    # Fixed: verify_checkpoint.py uses positional arg, not --checkpoint
    !python tools/verify_checkpoint.py checkpoints/best_model.pt
    
    ts = datetime.now().strftime('%Y%m%d_%H%M')
    shutil.make_archive(f'xtitan_model_{ts}', 'zip', '.', 'checkpoints')
    print(f'\n📦 Download: xtitan_model_{ts}.zip')
else:
    print('❌ No checkpoint found')
    print(f'   Existing: {list(Path("checkpoints").glob("*.pt"))}')

## 9. Quick Test

In [ ]:
if Path('checkpoints/best_model.pt').exists():
    import torch
    from config.settings import load_settings
    from models.core import DerivOmniModel
    
    settings = load_settings()
    model = DerivOmniModel(settings)
    # PyTorch 2.6+ requires weights_only=False for checkpoints with custom classes
    ckpt = torch.load('checkpoints/best_model.pt', map_location='cpu', weights_only=False)
    model.load_state_dict(ckpt['model_state_dict'])
    model.eval()
    
    with torch.no_grad():
        probs = model.predict_probs(
            torch.randn(1, settings.data_shapes.sequence_length_ticks),
            torch.randn(1, settings.data_shapes.sequence_length_candles, settings.data_shapes.feature_dim_candles),
            torch.randn(1, settings.data_shapes.feature_dim_volatility)
        )
    print(f'✅ Inference OK: rise_fall={probs["rise_fall_prob"].item():.4f}')
else:
    print('⏭️ No model')